In [62]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import io
from skimage.measure import label as skimage_label
from skimage.metrics import (adapted_rand_error, contingency_table,
                              variation_of_information)
from skimage.segmentation import join_segmentations
from sklearn.cluster import KMeans
from skimage.filters import threshold_otsu
import warnings
warnings.filterwarnings("ignore")
import cv2

In [9]:
def show_compared_images(t1_image, segmented_image, text):
    fig, (ax1, ax3) = plt.subplots(1, 3, figsize=(15, 15))
    # Plot Original T1 Image
    img1 = ax1.imshow(t1_image)
    fig.colorbar(img1, ax=ax1, location='bottom',pad=0.01)
    ax1.set_title('Original T1 Image', fontsize=15)
    ax1.axis('off')

    # Plot Segmented Image
    img2 = ax2.imshow(segmented_image)
    fig.colorbar(img2, ax=ax2, location='bottom', pad=0.01)
    #text = str(n_cluster)
    ax2.set_title(text, fontsize=15)
    ax2.axis('off')

    plt.show()
    
def region_growing(image, seed_point, threshold):
    image =  cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    # Create a mask and initialize it with zeros
    mask = np.zeros_like(image, dtype=np.uint8)

    # Define the region growing criteria
    connectivity = 4  # 4-connected neighborhood
    new_value = 255   # Set the intensity value for the segmented region

    # Create a list to store the seed points
    seed_points = [seed_point]

    # Iterate over the seed points until the list is empty
    while seed_points:
        # Pop the first seed point from the list
        current_point = seed_points.pop(0)

        # Check if the current point is within the image bounds
        if (0 <= current_point[0] < image.shape[0]) and (0 <= current_point[1] < image.shape[1]):
            # Check if the current pixel is not already segmented
            if mask[current_point].any() == 0:
                # Check the intensity difference between the current pixel and the seed pixel
                if abs(int(image[current_point]) - int(image[seed_point])) < threshold:
                    # Add the current pixel to the segmented region
                    mask[current_point] = new_value

                    # Add the neighboring pixels to the seed points list
                    seed_points.append((current_point[0] + 1, current_point[1]))
                    seed_points.append((current_point[0] - 1, current_point[1]))
                    seed_points.append((current_point[0], current_point[1] + 1))
                    seed_points.append((current_point[0], current_point[1] - 1))

    return mask


def process_images_region_growing(s1, s2, thresh, imgs):
    seed_point = (s1, s2)
    threshold = thresh
    for i in imgs:
        # Perform region growing segmentation
        segmented_image = region_growing(i, seed_point, threshold)
        show_compared_images(i, segmented_image, 'Region Growing Segmentation')
        #print('Level  '+ str(i) + ' Image')
        #eval_matrix(thresholded_image, label[:,:,i])
        #print(evaluate_segmentation(segmented_image, label[:,:,i]))

# Cluserting Based: K-Mean segmentation
def kmeans_segmentation(image, n_clusters=5):
    image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    reshaped_image = np.reshape(image, (-1, 1))

    #X_normalised = (reshaped_image - np.mean(reshaped_image))/np.std(reshaped_image)
    #print(image)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0, max_iter = 50, n_init=10).fit(reshaped_image)
    segmented_image = np.reshape(kmeans.labels_, image.shape)
    return segmented_image

def process_images_k_mean(n_cluster):
    for i in range(10):
        segmented_image = kmeans_segmentation(i, n_cluster)
        show_compared_images(i, segmented_image, 'K-Mean ' + str(n_cluster )+ ' Cluster')
        #print('Level  '+ str(i) + ' Image')
        #eval_matrix(segmented_image, label[:,:,i])
        #print(evaluate_segmentation(segmented_image, label[:,:,i]))

In [ ]:
new_img = img = io.imread('Img2.jpeg')
normalized_image = cv2.normalize(new_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
for i in range(1,10):
    n_cluster = i
    blur = cv2.GaussianBlur(normalized_image, (3,3), 0)
    #print('blur: ', blur.shape)     
    #plt.imshow(img1)
    # Reshaping the image into a 2D array of pixels and 3 color values (RGB) 
    pixel_vals = blur.reshape((-1,2)) # numpy reshape operation -1 unspecified 
    # Convert to float type only for supporting cv2.kmean
    pixel_vals = np.float32(pixel_vals)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 120, 0.90) 
    
    # Choosing number of cluster
    k = n_cluster
    
    retval, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS) 
    # convert data into 8-bit values 
    centers = np.uint8(centers) 
    segmented_data = centers[labels.flatten()] # Mapping labels to center points( RGB Value)
    # reshape data into the original image dimensions 
    segmented_image = segmented_data.reshape((blur.shape))  
    #plt.imshow(segmented_image)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
    ax1.imshow(normalized_image)
    ax1.set_title('Original T1 Image',fontsize=15)
    ax1.axis('off')
    
    ax2.imshow(segmented_image)
    text = str(n_cluster) + ' K Clusters'
    ax2.set_title(text,fontsize=15)
    ax2.axis('off')

In [10]:
new_img = img = io.imread('Img1.jpeg')
normalized_image = cv2.normalize(new_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
seed_point = (50, 50)
threshold = 50
new_img = np.reshape(normalized_image, (-1, 1))
segmented_image = region_growing(new_img, seed_point, threshold)


In [20]:
import cv2
import torch
import numpy as np
# Load the model with map_location='cpu'
model = torch.load('SINet_40.pth', map_location=torch.device('cpu'))

# model.eval()

# Preprocess the image
def preprocess_image(image):
    # Resize the image to match SINet input size
    resized_image = cv2.resize(image, (512, 512))
    # Convert to tensor and normalize
    tensor_image = torch.tensor(resized_image, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return tensor_image.unsqueeze(0)

# Perform inference
def predict(image):
    with torch.no_grad():
        output = model(image)
    return output

# Load the input image
image = cv2.imread('Img1.jpeg')

# Preprocess the image
preprocessed_image = preprocess_image(image)

# Perform inference
output = predict(preprocessed_image)

# Post-process the predictions (example)
def post_process(output):
    # Perform thresholding
    thresholded_output = (output > 0.5).float()
    return thresholded_output

# Post-process the output
processed_output = post_process(output)

# Visualize the results
# (You need to implement visualization based on the processed output)

# Display the original image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
ax1.imshow(image)
ax1.set_title('Original T1 Image',fontsize=15)
ax1.axis('off')

ax2.imshow(image)
text = 'Camouflaged Image'
ax2.set_title(text,fontsize=15)
ax2.axis('off')

print('execution completed')


TypeError: 'collections.OrderedDict' object is not callable

In [63]:
images = ['Img1.jpeg', 'Img2.jpeg','Img3.jpeg','Img4.jpeg','Img5.jpeg',
      'Img6.jpeg','Img7.jpeg','Img8.jpeg', 'Img9.jpeg', 'Img10.jpeg']
images2 = ['Sod/Img1.JPG', 'Sod/Img2.JPG','Sod/Img1.JPG', 'Sod/Img1.JPG',
           'Sod/Img1.JPG','Sod/Img1.JPG','Sod/Img1.JPG','Sod/Img1.JPG',
           'Sod/Img1.JPG','Sod/Img1.JPG','Sod/Img1.JPG','Sod/Img1.JPG','Sod/Img1.JPG','Sod/Img1.JPG',]

In [12]:
#This gave some results
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
for img in images:
    
    img = read_image(img)
    
    # Step 1: Initialize model with the best available weights
    weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
    FasterRCNN_ResNet50_FPN_V2_Weights.get_state_dict
    model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
    model.eval()
    
    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()
    
    # Step 3: Apply inference preprocessing transforms
    batch = [preprocess(img)]
    
    # Step 4: Use the model and visualize the prediction
    prediction = model(batch)[0]
    labels = [weights.meta["categories"][i] for i in prediction["labels"]]
    box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                              labels=labels,
                              colors="red",
                              width=4, font_size=30)
    im = to_pil_image(box.detach())
    im.show()

In [65]:
import torch
for i  in range(1,44):
    #print('Sod/img'+str(i)+'.JPG')
    img = read_image('Sod/img'+str(i)+'.JPG')

    
    # Step 1: Initialize model with the best available weights
    weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
    model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.75)
    model.eval()
    
    # Step 2: Initialize the inference transforms
    preprocess = weights.transforms()
    
    # Step 3: Apply inference preprocessing transforms
    batch = [preprocess(img)]

    # Step 4: Use the model and visualize the prediction
    # with torch.no_grad():
    #     prediction = model(batch)[0]
    # # Step 4: Use the model and visualize the prediction
    prediction = model(batch)[0]
    # for i in prediction["labels"]:
    #     if i.item() == 1:
    #         print('i: ', i.item())
    
    # labels = [weights.meta["categories"][i] for i in prediction["labels"] if i.item() == 1]

    # #labels = [i for i in labels if i == 'person']
    # box = draw_bounding_boxes(img, boxes=prediction["boxes"],
    #                           labels=labels,
    #                           colors="red",
    #                           width=4, font_size=30)

    # Convert the list of dictionaries to a list of bounding boxes
    person_indi = prediction['labels'] == 1
    person_boxes = prediction['boxes'][person_indi]
    #person_boxes = [box['boxes'] for box in prediction]

    # Visualize the 'person' boxes on the original image
    box = draw_bounding_boxes(img, boxes=person_boxes,
                            labels=['person']*len(person_boxes),  # Only person label
                            colors="red",
                            width=4, font_size=30)

    im = to_pil_image(box.detach())
    #im.show()
    im.save('fastrcnn_rest50_result/Img'+str(i)+'.jpg')

In [18]:
from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.transforms.functional import to_pil_image

img = read_image("Img10.jpeg")

# Step 1: Initialize model with the best available weights
weights = FCN_ResNet50_Weights.DEFAULT
model = fcn_resnet50(weights=weights)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Step 3: Apply inference preprocessing transforms
batch = preprocess(img).unsqueeze(0)

# Step 4: Use the model and visualize the prediction
prediction = model(batch)["out"]
normalized_masks = prediction.softmax(dim=1)
class_to_idx = {cls: idx for (idx, cls) in enumerate(weights.meta["categories"])}
mask = normalized_masks[0, class_to_idx["dog"]]
to_pil_image(mask).show()

In [19]:
# Apply image detector on a single image.
detector = hub.load("https://www.kaggle.com/models/tensorflow/efficientdet/frameworks/TensorFlow2/variations/d0/versions/1")
detector_output = detector(image_tensor)
class_ids = detector_output["detection_classes"]


NameError: name 'hub' is not defined

In [3]:
import cv2
import torch
import numpy as np
# Load the model with map_location='cpu'
model = torch.load('checkpoint.pth', map_location=torch.device('cpu'))

# Load the input image
image = cv2.imread('Img1.jpeg')


In [7]:

# Assuming `model` is an OrderedDict
for key, value in model.items():
    print(f'Parameter: {key}')
    # print(value)

Parameter: module.shared_encoder.conv1.weight
Parameter: module.shared_encoder.bn1.weight
Parameter: module.shared_encoder.bn1.bias
Parameter: module.shared_encoder.bn1.running_mean
Parameter: module.shared_encoder.bn1.running_var
Parameter: module.shared_encoder.bn1.num_batches_tracked
Parameter: module.shared_encoder.layer1.0.conv1.weight
Parameter: module.shared_encoder.layer1.0.bn1.weight
Parameter: module.shared_encoder.layer1.0.bn1.bias
Parameter: module.shared_encoder.layer1.0.bn1.running_mean
Parameter: module.shared_encoder.layer1.0.bn1.running_var
Parameter: module.shared_encoder.layer1.0.bn1.num_batches_tracked
Parameter: module.shared_encoder.layer1.0.conv2.weight
Parameter: module.shared_encoder.layer1.0.bn2.weight
Parameter: module.shared_encoder.layer1.0.bn2.bias
Parameter: module.shared_encoder.layer1.0.bn2.running_mean
Parameter: module.shared_encoder.layer1.0.bn2.running_var
Parameter: module.shared_encoder.layer1.0.bn2.num_batches_tracked
Parameter: module.shared_enc